# Advent of Code 2021
## [Day 23: Amphipod](https://adventofcode.com/2021/day/23)

In [3]:
import numpy as np

In [81]:
import heapq

In [8]:
import html
html_formatter = get_ipython().display_formatter.formatters['text/html']

def ndarray_to_html(a):
    if a.dtype == '<U1':
        s = '\n'.join([''.join(line) for line in a])
    else:
        s = str(a)
    return "<pre>{}</pre>".format(html.escape(s))

html_formatter.for_type(np.ndarray, ndarray_to_html)
pass

In [4]:
import aocd
input_data = aocd.get_data(year=2021, day=23).split('\n')
input_data

['#############',
 '#...........#',
 '###D#D#A#A###',
 '  #C#C#B#B#',
 '  #########']

In [5]:
test_data = [
    '#############',
    '#...........#',
    '###B#C#B#D###',
    '  #A#D#C#A#',
    '  #########',
]

In [9]:
def parse_burrow(data):
    max_len = max(len(line) for line in data)
    data[3] = data[3].ljust(max_len)
    data[4] = data[4].ljust(max_len)
    burrow = np.array([list(line) for line in data])
    return burrow

burrow = parse_burrow(test_data)
burrow

array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['#', '#', '#', 'B', '#', 'C', '#', 'B', '#', 'D', '#', '#', '#'],
       [' ', ' ', '#', 'A', '#', 'D', '#', 'C', '#', 'A', '#', ' ', ' '],
       [' ', ' ', '#', '#', '#', '#', '#', '#', '#', '#', '#', ' ', ' ']],
      dtype='<U1')

In [32]:
step_cost = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000
}
burrow[3,5], step_cost[burrow[3,5]]

('D', 1000)

In [84]:
home_column = {
    'A': 3,
    'B': 5,
    'C': 7,
    'D': 9
}

In [59]:
def move_cost(src, dst, species):
    # if the move is legal, cost is manhattan distance
    y1, x1 = src
    y2, x2 = dst
    if x1 == x2:
        steps = abs(y1 - y2)
    else:
        steps = abs(y1 - 1) + abs(x2 - x1) + abs(y2 - 1)
    return step_cost[species] * steps
move_cost((3,5), (3,3), 'B')

60

In [61]:
move_cost((1,1), (3,3), 'A')

4

general strategy is to move "D" blockers out, then move "D"s in.

then proceed to earlier letters.

don't need any state other than the board.

could use a recursive goal-driven approach:
- select a D
- find path to its home
- move to hall
    - for each blocker along the path:
        - recursively move it toward its home
- move within hall, if permitted
- move to home
    - for each blocker along the path:
        - recursively move it toward its home


could search all state space:

recursive function to find lowest cost to win from current state

enumerate all legal moves from current state and their costs

boom

In [192]:
from functools import cached_property, total_ordering
from collections import deque

In [ ]:
class Burrow(object):
    def __init__(self, data, cost=0):
        self.cost = cost
        if isinstance(data, list):
            self.grid = parse_burrow(data)
        elif isinstance(data, Burrow):
            self.grid = data.grid.copy()
            
    def __str__(self):
        return '\n'.join([''.join(line) for line in self.grid])
    
    def __repr__(self):
        return str(self) + f" unsolved: {self.num_unsolved}, cost: {self.cost}"
    
    def __hash__(self):
        return hash(str(self))
    
    def __eq__(self, other: "Burrow"):
        return np.array_equal(self.grid, other.grid)
    
    def __getitem__(self, idx):
        return self.grid[tuple(idx)]

    def __setitem__(self, idx, value):
        self.grid[tuple(idx)] = value
        
    def score(self):
        return self.num_unsolved * 100000 + self.cost
    
    def __lt__(self, rhs: "Burrow"):
        return self.score() < rhs.score()

    @cached_property
    def num_unsolved(self):
        total_count = 0
        for species, home_col in home_column.items():
            mismatch = False
            for tile in reversed(self.grid[2:-1,home_col]):
                if (tile != species) or mismatch:
                    mismatch = True
                    total_count += 1
        return total_count
    
    def is_solved(self):
        return self.num_unsolved == 0
    
    def legal_moves(self):
        moves = []
        for loc in self.amphipod_locations():
            moves += self.legal_moves_from(loc)
        return moves
    
    def legal_moves_from(self, src):
        species = self[src]
        s_row, s_col = src
        legal_moves = []
        for dst, cost in self.possible_moves_from(src).items():
            d_row, d_col = dst
            if d_row == 1 and d_col in [3,5,7,9]:
                continue
            if d_row == 1 and s_row == 1:
                continue
            if d_col == s_col:
                continue
            new_burrow = Burrow(self, self.cost + cost)
            new_burrow[dst] = species
            new_burrow[src] = '.'
            if new_burrow.num_unsolved > self.num_unsolved:
                continue
            legal_moves.append(new_burrow)
        return legal_moves
    
    def possible_moves_from(self, src):
        species = self[src]
        directions = np.array([[1,0],[-1,0],[0,-1],[0,1]])
        seen = {tuple(src): 0}
        to_check = deque([(np.array(src), 0)])
        while to_check:
            checking, cost = to_check.popleft()
            for d in directions:
                dst = checking + d
                if self[dst] != '.':
                    continue
                if tuple(dst) not in seen:
                    seen[tuple(dst)] = cost + step_cost[species]
                    to_check.append((dst, cost + step_cost[species]))
        del seen[tuple(src)]
        return seen

    def amphipod_locations(self):
        return np.array(np.where(self.grid >= 'A')).T

b = Burrow(test_data)
#b.legal_moves_from(np.array([2,3]))
#for loc in b.amphipod_locations():
#    print(loc, len(b.legal_moves_from(loc)))
print(len(b.legal_moves()))
b

In [205]:
b

#############
#...........#
###B#C#B#D###
  #A#D#C#A#  
  #########   cost: 0

In [163]:
c = Burrow(input_data)
c

#############
#...........#
###D#D#A#A###
  #C#C#B#B#  
  #########   cost: 0

In [164]:
sorted([b, c], key=lambda x: x.num_unsolved)

[#############
 #...........#
 ###B#C#B#D###
   #A#D#C#A#  
   #########   cost: 0,
 #############
 #...........#
 ###D#D#A#A###
   #C#C#B#B#  
   #########   cost: 0]

In [169]:
{b: b.cost,c: c.cost}

{#############
 #...........#
 ###B#C#B#D###
   #A#D#C#A#  
   #########   cost: 0: 0,
 #############
 #...........#
 ###D#D#A#A###
   #C#C#B#B#  
   #########   cost: 0: 0}

In [153]:
b.legal_moves()

[#############
 #..B........#
 ###.#C#B#D###
   #A#D#C#A#  
   #########   cost: 10,
 #############
 #....C......#
 ###B#.#B#D###
   #A#D#C#A#  
   #########   cost: 100,
 #############
 #......B....#
 ###B#C#.#D###
   #A#D#C#A#  
   #########   cost: 10,
 #############
 #........D..#
 ###B#C#B#.###
   #A#D#C#A#  
   #########   cost: 1000]

In [93]:
[tile for tile in reversed(b.grid[2:-1,3])]

['A', 'B']

In [76]:
src = tuple(np.array(np.where(b.grid >= 'A')).T[0])
src

(2, 3)

In [77]:
b.grid[src]

'B'

In [156]:
solved_data = [
    '#############',
    '#...........#',
    '###A#B#C#D###',
    '  #A#B#C#D#',
    '  #########',
]
Burrow(solved_data).is_solved()

True

In [158]:
from heapq import heapify, heappush, heappop

In [278]:
def find_best_solution(b: burrow):
    to_check = [b]
    heapify(to_check)
    seen_states = {b: b.cost}
    best_cost = 9999999
    i = 0
    while to_check:
        if i % 5000 == 0:
            print(f"iteration {i}: to check: {len(to_check)}")
        i += 1
        checking = heappop(to_check)
        if checking.cost > best_cost:
            continue
        if seen_states[checking] < checking.cost:
            continue
        for move in checking.legal_moves():
            if move.is_solved():
                if move.cost < best_cost:
                    best_cost = move.cost
                    print(f"found solution with cost {move.cost}")
            if move in seen_states and seen_states[move] <= move.cost:
                continue
            seen_states[move] = move.cost
            heappush(to_check, move)
    return best_cost

b = Burrow(input_data)
%time find_best_solution(b)

iteration 0: to check: 1
found solution with cost 16491
iteration 5000: to check: 2365
iteration 10000: to check: 4063
iteration 15000: to check: 4990
iteration 20000: to check: 1450
iteration 25000: to check: 2958
iteration 30000: to check: 3568
iteration 35000: to check: 2633
iteration 40000: to check: 1366
iteration 45000: to check: 1121
iteration 50000: to check: 1242
iteration 55000: to check: 2601
iteration 60000: to check: 1348
iteration 65000: to check: 1767
iteration 70000: to check: 2247
iteration 75000: to check: 2206
iteration 80000: to check: 5094
iteration 85000: to check: 6641
iteration 90000: to check: 9596
iteration 95000: to check: 10443
iteration 100000: to check: 11966
iteration 105000: to check: 13202
iteration 110000: to check: 12197
iteration 115000: to check: 8114
iteration 120000: to check: 11641
iteration 125000: to check: 12646
iteration 130000: to check: 12229
iteration 135000: to check: 8021
iteration 140000: to check: 8634
iteration 145000: to check: 8982


KeyboardInterrupt: 

#### Part 1 Answer
**What is the least energy required to organize the amphipods?**

In [ ]:
16491 # not the right answer